In [57]:

import numpy as np
import pickle
import matplotlib.pyplot as plt
from minisom import MiniSom  
from sklearn.cluster import KMeans

In [58]:
print("SOM Assignment")

SOM Assignment


In [59]:
chainlink = np.genfromtxt('chainlink.vec', skip_header=4, usecols=list(range(3)))

In [60]:
chainlink
print(chainlink.shape)
d1=20
d2=25


(1000, 3)


In [61]:
chainlink_som_small = MiniSom(20, 25, 3, sigma=20, learning_rate=0.1,neighborhood_function='triangle',random_seed=10)
chainlink_som_small.train_random(chainlink, 10000)
chainlink_som_big = MiniSom(100, 60, 3, sigma=20, learning_rate=0.5)
chainlink_som_big.train_random(chainlink, 10000)

C:\Users\Stefana Iosub\Miniconda3\lib\site-packages\minisom.py:145: UserWarning: Warning: sigma is too high for the dimension of the map.
  warn('Warning: sigma is too high for the dimension of the map.')
C:\Users\Stefana Iosub\Miniconda3\lib\site-packages\numpy\linalg\linalg.py:2512: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
C:\Users\Stefana Iosub\Miniconda3\lib\site-packages\numpy\linalg\linalg.py:2513: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
C:\Users\Stefana Iosub\Miniconda3\lib\site-packages\minisom.py:263: RuntimeWarning: invalid value encountered in add
  self._weights += einsum('ij, ijk->ijk', g, x-self._weights)


In [62]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec



In [63]:
# HIERARCHICAL CLUSTERING

In [64]:
from scipy import cluster


In [65]:
chainlink_som_small.train_random(chainlink, 1000)

print("Ev Small SOM")
print(chainlink_som_small.quantization_error(chainlink))
print(chainlink_som_small.topographic_error(chainlink))
print("Mean of Quantization error:")
print(chainlink_som_small.quantization_error(chainlink)/d1*d2)


print("Ev Big SOM")
print(chainlink_som_big.quantization_error(chainlink))
print(chainlink_som_big.topographic_error(chainlink))

print("Mean of Quantization error:")
print(chainlink_som_big.quantization_error(chainlink)/d1*d2)




weights=chainlink_som_small.get_weights()
weights = chainlink_som_small.get_weights().reshape(d1*d2,-1) 





def hierarchy(weights):
    Z = cluster.hierarchy.linkage(weights,'single')
    d=cluster.hierarchy.dendrogram(Z,truncate_mode='level')
    #Y=cluster.hierarchy.single(weights)
    plt.show()
    print(Z)

    cutree = cluster.hierarchy.cut_tree(Z, n_clusters=[2,10])
    labels=cutree[:,1]
    #print("Labeling",labels)

    #print(cutree)
    return labels

labels=hierarchy(weights)


Ev Small SOM
nan
0.578
Mean of Quantization error:
nan
Ev Big SOM


C:\Users\Stefana Iosub\Miniconda3\lib\site-packages\minisom.py:408: RuntimeWarning: overflow encountered in power
  weights_flat_sq = power(weights_flat, 2).sum(axis=1, keepdims=True)


0.010982796020281422
0.001
Mean of Quantization error:
0.013728495025351777


ValueError: The condensed distance matrix must contain only finite values.

In [ ]:
def winners_labels(som,weights,labels):
    winners_labels = np.zeros(shape=(d1,d2))
    for cnt, xx in enumerate (weights):
        w = som.winner(xx)
        l = labels[cnt]
        winners_labels[w[0],w[1]] = l
    return winners_labels

def dif_matrix(label_matrix):
    dif_mat = np.zeros(shape=label_matrix.shape)
    for i,row in enumerate(label_matrix):
        for j,label in enumerate(row):
            if i < label_matrix.shape[0]-1 and label_matrix[i][j] != label_matrix[i+1][j]:
                dif_mat[i][j] = 1
            elif j < label_matrix.shape[1]-1 and label_matrix[i][j] != label_matrix[i][j+1]:
                dif_mat[i][j] = 1
    return dif_mat

dif_matrix(winners_labels(chainlink_som_small,weights,labels))

print(np.unique(dif_matrix(winners_labels(chainlink_som_small,weights,labels))))

In [ ]:
## import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def plot_with_labelling(som, d1, d2, labels, coloring_labels, rweights, debug=False):

    plt.figure(figsize=(d1//2, d2//2))
    # Plotting the response for each pattern in the iris dataset
    #win_map = som.win_map(rweights)
    #plt.pcolor(som.distance_map().T, cmap='bone_r')  # plotting the distance map as background
    #plt.colorbar()
    # use different colors and markers for each label
    #markers = ['o', 's', 'D']
    colors = ['C' + str(i) for i in np.unique(coloring_labels)] #['C0', 'C1', 'C2', 'C3', 'C4']
    win_labels = winners_labels(som,rweights,labels)
    diff_mat_ = dif_matrix(win_labels)
    diff_mat = np.append(diff_mat_,np.ones((diff_mat_.shape[0],1)),axis=1)
    diff_mat = np.append(diff_mat,np.ones((1,diff_mat.shape[1])),axis=0)

    for cnt, xx in enumerate(rweights):
        w = som.winner(xx)  # getting the winner
        #w = xx
        # palce a marker on the winning position for the sample xx
        plt.plot(w[0]+.5, w[1]+.5, 's', markerfacecolor=colors[coloring_labels[cnt]],
                markeredgecolor=colors[coloring_labels[cnt]], markersize=20, markeredgewidth=2)
    def plot_borders(debug=False):
        border_color = lambda c : c if debug else 'black'
        for i,row in enumerate(diff_mat):
            for j,v in enumerate(row):
                if i < diff_mat.shape[0]-1 and j < diff_mat.shape[1]-1:
                    if v != 0: 
                        # case where
                        # - -  -
                        # -[1] 1
                        # - -  -
                        # then draw horizontal line
                        if i > 0 and diff_mat[i-1][j] != 0:
                            plt.plot([i,i+1],[j+1,j+1], color=border_color('red'))
                        # edge case where
                        # - - -
                        # [1] -
                        # - - -
                        # then draw horizontal line
                        elif i == 0 and j != 0:
                            plt.plot([i,i+1],[j+1,j+1], color=border_color('green'))    
                        # case where
                        # - 0  1
                        # -[1] 0
                        # - -  -
                        # then draw horizontal line
                        elif i > 0 and diff_mat[i-1][j] == 0 and diff_mat[i][j+1] == 0 and diff_mat[i-1][j+1] != 0:
                            plt.plot([i,i+1],[j+1,j+1], color=border_color('blue'))
                        # case where
                        # - 1  -
                        # -[1] -
                        # - -  -
                        # then draw vertical line
                        if diff_mat[i][j+1] != 0:
                            plt.plot([i+1,i+1],[j+1,j+2], color=border_color('black'))
                        # edge case where
                        # - [1] -
                        # -  -  -
                        # then draw vertical line
                        if j == 0 and i != 0:
                            plt.plot([i+1,i+1],[j,j+1], color=border_color('brown'))    
                    # case where
                    # - -  -
                    # -[0] 1
                    # - 1  0
                    # then draw vertical line
                    elif diff_mat[i+1][j] != 0 and diff_mat[i][j+1] != 0 and diff_mat[i+1][j+1] == 0:
                        plt.plot([i+1,i+1],[j+1,j+2], color=border_color('purple'))
    plot_borders(debug)
    plt.axis([0, d1, 0, d2])
    plt.show()

d1 = 20
d2 = 25
rweights = chainlink_som_small.get_weights().reshape(d1*d2,-1) # first argument is product of first two dimensions of the input dataset
Z = cluster.hierarchy.linkage(rweights,'ward')
labels = cluster.hierarchy.cut_tree(Z, n_clusters=2)
labels = [l[0] for l in labels]
plot_with_labelling(chainlink_som_small,d1,d2,labels,labels,rweights,debug=False)

In [ ]:
# K-Means clustering

In [ ]:


starting_weights = chainlink_som_small.get_weights()
print(starting_weights)
weightskM = chainlink_som_small.get_weights().reshape(d1*d2,-1) # first argument is product of first two dimensions of the input dataset

#x=weights[1]
#print(weights[x])
labels_kM = KMeans(n_clusters=2,random_state=0)
y_kmeans2 = labels_kM.fit(weightskM)
print("Y_Kmeans2",y_kmeans2)
#KMlabels=y_kmeans2.labels_

#print(KMlabels)


In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(weightskM)
labels_kM = kmeans.labels_
print(labels_kM)
plot_with_labelling(chainlink_som_small,d1,d2,labels_kM,labels_kM,rweights,debug=False)